### Carrega Processos Baixados do Tribunal no Banco

1. Carrega o arquivo CSV que foi baixado no passo "Encontra Processos"
2. Para Cada linha no arquivo, inclui/atualiza o documento na base 




In [1]:
# Importando as Bibliotecas
import pandas as pd
import json
import pymongo
import yaml

In [ ]:
# Carrega Configurações do arquivo config
with open("config.yaml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)

user = cfg["mongodb"]["user"]
passwd = cfg["mongodb"]["passwd"]
host = cfg["mongodb"]["host"]
dbmongo = cfg["mongodb"]["db"]

In [ ]:
# Cria Conexão com banco
url_conexao = "mongodb+srv://" + user + ":" + passwd + "@" + host + "/?retryWrites=true&w=majority"
client = pymongo.MongoClient(url_conexao)
db = client[dbmongo]


In [ ]:
# Coleções Mongo
col_processos = db['processo']


In [3]:
#Parâmetros de Carga
arquivo = "2022_09_22_tjsp_processos_mapfre.csv"


In [ ]:
# Carrega o arquivo CSV em um Data Frame
df_processos = pd.read_csv('./dados/' + arquivo,sep=';',encoding='utf-8')
df_processos

In [5]:
# Carrega os dados do processo na base, se já existe, atualiza. 
for index,proc in df_processos.iterrows(): 
    json_str = df_processos.loc[index].to_json()
    json_str = json_str.replace('[','')
    json_str = json_str.replace(']','')
    json_str = '{"processo":' + json_str + '}'
    col_processos.find_one_and_update({'processo.nro_processo':df_processos.loc[0]['nro_processo']},{'$set':json.loads(json_str)},upsert=True)    
    

    